In [2]:
#@title
mkdir A8
#@title
mv /content/drive/MyDrive/Colab\ Notebooks/Assignment_8_IR.ipynb A8
#@title
cd A8
#@title
ls
#@title
mkdir data
#@title
cd data
#@title
!gdown https://drive.google.com/uc?id=1q0ZimHCtMlhftljfVuy0i-wcgPnDUYjl
#@title
!unzip fake-news.zip

# codes

In [2]:
cd /content/drive/MyDrive/workspace/IR/A8

/content/drive/MyDrive/workspace/IR/A8


In [3]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv('data/train.csv')

In [37]:
dftest = pd.read_csv('data/test.csv')

In [38]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [7]:
class document:
  """
  custom class to represent comments
  """
  def __init__(self, id, text, label):
    """
    initialization of comments
    """
    self.id = id
    self.text = text
    self.label = label
  
  def read(self):
    """
    returns comment's text
    """
    return self.text

# Using titles

In [24]:
docs = [document(id, text, label) for id, text, label in np.array(df[['id', 'title', 'label']])]

In [25]:
docs_fit = df['title'].values.astype('U')

In [39]:
termtitleMat = TfidfVectorizer(docs, token_pattern='[a-zA-Z]+', stop_words={'english'}).fit_transform(docs_fit)

In [40]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import StackingClassifier

In [41]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
clf = MultinomialNB()

## Voting Classifier

In [53]:
voting_clf_title = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('clf', clf)], voting='hard')
voting_clf_title.fit(termdocMat, df['label'])


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [55]:
from sklearn.model_selection import cross_val_score

In [57]:
cross_val_score(voting_clf_title, termtitleMat, df['label'], cv=5)

array([0.93966346, 0.93774038, 0.93774038, 0.92956731, 0.93605769])

## Stacking classifier

In [58]:
stack_clf_title = StackingClassifier(estimators=[('rf', rnd_clf), ('clf', clf)], final_estimator=LogisticRegression())

In [59]:
stack_clf_title.fit(termtitleMat, df['label'])

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [60]:
cross_val_score(stack_clf_title, termtitleMat, df['label'], cv=5)

array([0.94591346, 0.95216346, 0.94615385, 0.93966346, 0.94375   ])

# Using text

In [62]:
docs = [document(id, text, label) for id, text, label in np.array(df[['id', 'text', 'label']])]

In [63]:
def gen(docs):
  for doc in docs:
    yield str(doc.text)
docs_fit = gen(docs)

In [64]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [65]:
termtextMat = TfidfVectorizer(docs, token_pattern='[a-zA-Z]+', stop_words={'english'}).fit_transform(docs_fit)

## Voting classifier

In [66]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
clf = MultinomialNB()

In [67]:
voting_clf_text = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('clf', clf)], voting='hard')
voting_clf_text.fit(termtextMat, df['label'])

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [68]:
cross_val_score(voting_clf_text, termtextMat, df['label'], cv=5)

array([0.92860577, 0.93052885, 0.94038462, 0.93581731, 0.93317308])

## Stacking Classifier

In [69]:
stack_clf_text = StackingClassifier(estimators=[('rf', rnd_clf), ('clf', clf)], final_estimator=LogisticRegression())

In [70]:
stack_clf_text.fit(termtextMat, df['label'])

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [71]:
cross_val_score(stack_clf_text, termtextMat, df['label'], cv=5)

array([0.93557692, 0.93557692, 0.940625  , 0.94302885, 0.93461538])

**Observations**

The performance of all the classifiers based on titles and text were almost same. Here are the scores I got using cross validation on training data.

In [8]:
print(f'Using titles only: votingClassifier -> {np.array([0.93966346, 0.93774038, 0.93774038, 0.92956731, 0.93605769])} \n avg accuracy:{np.mean(np.array([0.93966346, 0.93774038, 0.93774038, 0.92956731, 0.93605769]))}')
print(f'Using titles only: StackingClassifier -> {np.array([0.94591346, 0.95216346, 0.94615385, 0.93966346, 0.94375   ])} \n avg accuracy:{np.mean(np.array([0.94591346, 0.95216346, 0.94615385, 0.93966346, 0.94375   ]))}')
print(f'Using text : votingClassifier -> {np.array([0.92860577, 0.93052885, 0.94038462, 0.93581731, 0.93317308])} \n avg accuracy:{np.mean(np.array([0.92860577, 0.93052885, 0.94038462, 0.93581731, 0.93317308]))}')
print(f'Using text : stackingClassifier -> {np.array([0.93557692, 0.93557692, 0.940625  , 0.94302885, 0.93461538])} \n avg accuracy:{np.mean(np.array([0.93557692, 0.93557692, 0.940625  , 0.94302885, 0.93461538]))}')


Using titles only: votingClassifier -> [0.93966346 0.93774038 0.93774038 0.92956731 0.93605769] 
 avg accuracy:0.936153844
Using titles only: StackingClassifier -> [0.94591346 0.95216346 0.94615385 0.93966346 0.94375   ] 
 avg accuracy:0.9455288459999999
Using text : votingClassifier -> [0.92860577 0.93052885 0.94038462 0.93581731 0.93317308] 
 avg accuracy:0.933701926
Using text : stackingClassifier -> [0.93557692 0.93557692 0.940625   0.94302885 0.93461538] 
 avg accuracy:0.937884614


Stacking classifier using titles only performed best with avg 94.55 % accuracy on all folds of cross validation. 